## Projeto Final - Etapa 1

Aluna: Jaisa O. Nichterwitz

In [16]:
%pip install plyer
%pip install tqdm
%pip install requests



Note: you may need to restart the kernel to use updated packages.


In [17]:
## importação das bibliotecas necessárias para o projeto :)

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

Função Alerta utilizando a aula de Fundamentos de programação III


In [18]:
# Função de alerta por nível conforme aprendido na aula 4:

def alerta(nivel, base, etapa, erro=""):
    '''
        Alerta de falha de carregamento de dados
    '''
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    # Determinar título do alerta baseado no nível
    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print(f"Nível {nivel} não disponível!")
        return

    # Notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

Banco de Dados

In [19]:
def tabelas_bd():
    '''
        Retorna um dataframe com as tabelas do banco de dados.
    '''
    conn = sqlite3.connect('restcountries.db')
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)
    conn.close()
    return schema

def salva_bd(df, nome_tabela):
    '''
        Salva o dataframe na tabela especificada.
    '''
    conn = sqlite3.connect('restcountries.db')
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    conn.close()
    return True

def carrega_bd(nome_tabela):
    '''
        Carrega a tabela especificada em um dataframe.
    '''
    conn = sqlite3.connect('restcountries.db')
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)
    conn.close()
    return df


Extração de Tabelas

In [20]:
def get_json_api(url):
    '''
        Request GET url e retorna json de saida
    '''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    
    # Erro
    raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")


Tabela 1: Países

In [21]:
def get_base_countries():
    '''
        Extrai dados completos e retorna um DataFrame com nome, capital e população dos países.
    '''
    # URL
    url = "https://restcountries.com/v3.1/all"
    
    # Coleta dados JSON da API
    json_data = get_json_api(url)
    
    # Cria dataframe com as colunas (nome, capital, população)
    df_countries = pd.DataFrame({
        "name": [country.get('name', {}).get('common', 'N/A') for country in json_data],
        "capital": [country.get('capital', ['N/A'])[0] for country in json_data],
        "population": [country.get('population', 'N/A') for country in json_data]
    })
    
    return df_countries

# Amostra da tabela
df_countries = get_base_countries()
print("Tabela de Países:")
print(df_countries.head())


Tabela de Países:
            name            capital  population
0  South Georgia  King Edward Point          30
1        Grenada       St. George's      112519
2    Switzerland               Bern     8654622
3   Sierra Leone           Freetown     7976985
4        Hungary           Budapest     9749763


Tabela 2: Regiões

In [22]:
# Função para extrair dados da tabela de regiões e sub-regiões
def get_base_regions():
    '''
        Extrai dados completos e retorna um DataFrame com a região e sub-região de cada país.
    '''
    url = "https://restcountries.com/v3.1/all"
    json_data = get_json_api(url)
    
    df_regions = pd.DataFrame({
        "name": [country.get('name', {}).get('common', 'N/A') for country in json_data],
        "region": [country.get('region', 'N/A') for country in json_data],
        "subregion": [country.get('subregion', 'N/A') for country in json_data]
    })
    
    return df_regions


# Amostra da tabela
df_regions = get_base_regions()
print("Tabela de Regiões e Sub-regiões:")
print(df_regions.head(), "\n")

Tabela de Regiões e Sub-regiões:
            name     region       subregion
0  South Georgia  Antarctic             N/A
1        Grenada   Americas       Caribbean
2    Switzerland     Europe  Western Europe
3   Sierra Leone     Africa  Western Africa
4        Hungary     Europe  Central Europe 



Tabela 3: Idiomas 

In [23]:
# Função para extrair dados da tabela de idiomas oficiais
def get_base_languages():
    '''
        Extrai dados completos e retorna um DataFrame com o idioma oficial de cada país.
    '''
    url = "https://restcountries.com/v3.1/all"
    json_data = get_json_api(url)
    
    df_languages = pd.DataFrame({
        "name": [country.get('name', {}).get('common', 'N/A') for country in json_data],
        "languages": [', '.join(country.get('languages', {}).values()) if 'languages' in country else 'N/A' for country in json_data]
    })
    
    return df_languages

df_languages = get_base_languages()
print("Tabela de Idiomas Oficiais:")
print(df_languages.head())

Tabela de Idiomas Oficiais:
            name                               languages
0  South Georgia                                 English
1        Grenada                                 English
2    Switzerland  French, Swiss German, Italian, Romansh
3   Sierra Leone                                 English
4        Hungary                               Hungarian


Extração completa

In [24]:
def etapa_extracao():
    
    #################################################
    print("  > Base de Países (Nome, Capital, População):\n")
    try:
        df_countries = get_base_countries()
        salva_bd(df_countries, "countries")
    except Exception as e:
        alerta(nivel=3, 
               base="countries", 
               etapa="EXTRACAO", 
               erro=e)
    
    #################################################
    print("  > Base de Regiões e Sub-Regiões:\n")
    try:
        df_regions = get_base_regions()
        salva_bd(df_regions, "regions")
    except Exception as e:
        alerta(nivel=3, 
               base="regions", 
               etapa="EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base de Idiomas Oficiais:\n")
    try:
        df_languages = get_base_languages()
        salva_bd(df_languages, "languages")
    except Exception as e:
        alerta(nivel=3, 
               base="languages", 
               etapa="EXTRACAO", 
               erro=e)
